In [16]:
import numpy as np 
import tensorflow as tf
from tensorflow import keras

In [17]:
file = open("test_data.txt")
data = file.read()

In [39]:
data

'*This Etext has certain copyright implications you should read!*\n\n<<THIS ELECTRONIC VERSION OF THE COMPLETE WORKS OF WILLIAM\nSHAKESPEARE IS COPYRIGHT 1990-1993 BY WORLD LIBRARY, INC., AND IS\nPROVIDED BY PROJECT GUTENBERG ETEXT OF ILLINOIS BENEDICTINE COLLEGE\nWITH PERMISSION.  ELECTRONIC AND MACHINE READABLE COPIES MAY BE\nDISTRIBUTED SO LONG AS SUCH COPIES (1) ARE FOR YOUR OR OTHERS\nPERSONAL USE ONLY, AND (2) ARE NOT DISTRIBUTED OR USED\nCOMMERCIALLY.  PROHIBITED COMMERCIAL DISTRIBUTION INCLUDES BY ANY\nSERVICE THAT CHARGES FOR DOWNLOAD TIME OR FOR MEMBERSHIP.>>\n\n*Project Gutenberg is proud to cooperate with The World Library*\nin the presentation of The Complete Works of William Shakespeare\nfor your reading for education and entertainment.  HOWEVER, THIS\nIS NEITHER SHAREWARE NOR PUBLIC DOMAIN. . .AND UNDER THE LIBRARY\nOF THE FUTURE CONDITIONS OF THIS PRESENTATION. . .NO CHARGES MAY\nBE MADE FOR *ANY* ACCESS TO THIS MATERIAL.  YOU ARE ENCOURAGED!!\nTO GIVE IT AWAY TO ANYONE

In [18]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(oov_token='missing')

In [19]:
tokenizer.fit_on_texts([data])

In [20]:
tokenizer.get_config()

{'num_words': None,
 'filters': '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
 'lower': True,
 'split': ' ',
 'char_level': False,
 'oov_token': 'missing',
 'document_count': 1,
 'word_counts': '{"this": 122, "etext": 25, "has": 3, "certain": 2, "copyright": 10, "implications": 1, "you": 145, "should": 47, "read": 8, "electronic": 7, "version": 4, "of": 402, "the": 472, "complete": 8, "works": 10, "william": 7, "shakespeare": 10, "is": 181, "1990": 3, "1993": 3, "by": 113, "world": 34, "library": 9, "inc": 5, "and": 514, "provided": 4, "project": 18, "gutenberg": 16, "illinois": 5, "benedictine": 5, "college": 5, "with": 178, "permission": 3, "machine": 4, "readable": 6, "copies": 8, "may": 42, "be": 151, "distributed": 6, "so": 145, "long": 17, "as": 121, "such": 37, "1": 7, "are": 77, "for": 190, "your": 110, "or": 121, "others": 11, "personal": 3, "use": 17, "only": 9, "2": 7, "not": 162, "used": 9, "commercially": 3, "prohibited": 3, "commercial": 3, "distribution": 4, "includes": 3, "an

In [21]:
n_classes = len(tokenizer.word_index)

In [22]:
input_sentence = []
for sentence in data.split('.'):
    sentence.strip()
    tokenized_sentence = tokenizer.texts_to_sequences([sentence])[0]
    
    for i in range(len(tokenized_sentence)):
        input_sentence.append(tokenized_sentence[:i+1])

In [23]:
input_sentence

[[24],
 [24, 107],
 [24, 107, 672],
 [24, 107, 672, 946],
 [24, 107, 672, 946, 229],
 [24, 107, 672, 946, 229, 1490],
 [24, 107, 672, 946, 229, 1490, 21],
 [24, 107, 672, 946, 229, 1490, 21, 61],
 [24, 107, 672, 946, 229, 1490, 21, 61, 280],
 [24, 107, 672, 946, 229, 1490, 21, 61, 280, 24],
 [24, 107, 672, 946, 229, 1490, 21, 61, 280, 24, 321],
 [24, 107, 672, 946, 229, 1490, 21, 61, 280, 24, 321, 530],
 [24, 107, 672, 946, 229, 1490, 21, 61, 280, 24, 321, 530, 5],
 [24, 107, 672, 946, 229, 1490, 21, 61, 280, 24, 321, 530, 5, 3],
 [24, 107, 672, 946, 229, 1490, 21, 61, 280, 24, 321, 530, 5, 3, 281],
 [24, 107, 672, 946, 229, 1490, 21, 61, 280, 24, 321, 530, 5, 3, 281, 230],
 [24, 107, 672, 946, 229, 1490, 21, 61, 280, 24, 321, 530, 5, 3, 281, 230, 5],
 [24,
  107,
  672,
  946,
  229,
  1490,
  21,
  61,
  280,
  24,
  321,
  530,
  5,
  3,
  281,
  230,
  5,
  322],
 [24,
  107,
  672,
  946,
  229,
  1490,
  21,
  61,
  280,
  24,
  321,
  530,
  5,
  3,
  281,
  230,
  5,
  322,
  2

In [24]:
max_length = max(len(x) for x in input_sentence)

In [25]:
max_length

134

In [26]:
from keras.utils import pad_sequences

In [27]:
padded_sequences = pad_sequences(input_sentence, maxlen=max_length)

In [28]:
input_x = padded_sequences[:,:-1]

In [29]:
input_y = padded_sequences[:,-1]

In [30]:
input_y = keras.utils.to_categorical(input_y, num_classes=n_classes+1)

In [31]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

In [32]:
model = Sequential([
    Embedding(n_classes+1, 100, input_length=max_length-1),
    LSTM(units=200),
    Dense(n_classes+1, activation='softmax')
])

In [33]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 133, 100)          359700    
                                                                 
 lstm (LSTM)                 (None, 200)               240800    
                                                                 
 dense (Dense)               (None, 3597)              722997    
                                                                 
Total params: 1323497 (5.05 MB)
Trainable params: 1323497 (5.05 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [34]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [35]:
model.fit(input_x, input_y,epochs=100)

Epoch 1/100
579/579 [==============================] - 73s 123ms/step - loss: 6.8619 - accuracy: 0.0272
Epoch 2/100
579/579 [==============================] - 72s 125ms/step - loss: 6.4398 - accuracy: 0.0366
Epoch 3/100
579/579 [==============================] - 74s 128ms/step - loss: 6.2239 - accuracy: 0.0467
Epoch 4/100
579/579 [==============================] - 74s 128ms/step - loss: 5.9700 - accuracy: 0.0562
Epoch 5/100
579/579 [==============================] - 73s 127ms/step - loss: 5.6588 - accuracy: 0.0694
Epoch 6/100
579/579 [==============================] - 73s 127ms/step - loss: 5.3017 - accuracy: 0.0831
Epoch 7/100
579/579 [==============================] - 75s 129ms/step - loss: 4.9152 - accuracy: 0.1024
Epoch 8/100
579/579 [==============================] - 74s 128ms/step - loss: 4.5159 - accuracy: 0.1320
Epoch 9/100
579/579 [==============================] - 75s 129ms/step - loss: 4.1224 - accuracy: 0.1795
Epoch 10/100
579/579 [==============================] - 74s 127m

In [48]:
input_str = "He loves"

for i in range(10):

    tokenized_input = tokenizer.texts_to_sequences([input_str])

    padded_input = pad_sequences(tokenized_input, maxlen=max_length-1)

    result = model.predict(padded_input)

    output_token = np.argmax(result[0])

    for word, index in tokenizer.word_index.items():
        if index == output_token:
            input_str = input_str + " " + word
            print(input_str)

1/1 [==============================] - 0s 27ms/step
He loves is
1/1 [==============================] - 0s 23ms/step
He loves is my
1/1 [==============================] - 0s 23ms/step
He loves is my verse
1/1 [==============================] - 0s 20ms/step
He loves is my verse so
1/1 [==============================] - 0s 16ms/step
He loves is my verse so barren
1/1 [==============================] - 0s 15ms/step
He loves is my verse so barren of
1/1 [==============================] - 0s 19ms/step
He loves is my verse so barren of new
1/1 [==============================] - 0s 12ms/step
He loves is my verse so barren of new pride
1/1 [==============================] - 0s 16ms/step
He loves is my verse so barren of new pride so
1/1 [==============================] - 0s 23ms/step
He loves is my verse so barren of new pride so long


In [37]:
result[0][2]

0.02140682